In [1]:
import os
import numpy as np
import pandas as pd

In [3]:
train = pd.DataFrame()
for name in os.listdir('proc_data/total')[:-1]:
    ins_data = pd.read_csv(f'proc_data/total/{name}', parse_dates=['date'])
    train = pd.concat([train, ins_data], axis=0).reset_index(drop=True)

train.sample(5)

,date,Np,Tp,Vp,B_gsm_x,B_gsm_y,B_gsm_z,Bmag,target
6956,2001-05-19 12:00:00,-292.633970,75559.340237,124.325740,-1.757651,6.550598,1.307988,7.093568,3
40767,2012-12-13 21:00:00,-9999.900000,-9999.900000,-393.471598,0.751077,-0.218905,1.011876,3.103154,0
13968,2003-10-13 00:00:00,-9999.900000,-9999.900000,-33.662249,-0.424249,3.610118,-2.266391,5.145805,3
32102,2009-12-26 18:00:00,-470.336107,51636.157988,-37.914675,-2.787065,1.446254,0.940734,3.791266,0
21563,2006-05-19 09:00:00,-354.306982,187709.288690,229.764821,-3.843202,0.098030,1.491012,5.028982,0


In [4]:
test = pd.read_csv(f"proc_data/total/{os.listdir('proc_data/total')[-1]}", parse_dates=['date'])

test.sample(5)

,date,Np,Tp,Vp,B_gsm_x,B_gsm_y,B_gsm_z,Bmag,target
1592,2013-07-19 00:00:00,-1662.894470,169743.704167,-937.419048,9.428065,-1.052851,-4.025893,10.919548,3
92,2013-01-12 12:00:00,-704.270698,42496.539645,-311.317160,-2.660243,1.744497,1.353562,5.214775,1
2170,2013-09-29 06:00:00,-9999.900000,-9999.900000,-215.501124,-1.138071,1.218408,0.265391,1.930982,0
2868,2013-12-25 12:00:00,-9999.900000,-9999.900000,-509.067160,0.688846,8.459065,-6.768450,10.945071,3
1456,2013-07-02 00:00:00,-593.204655,38903.939881,10.834405,-0.408452,1.866345,1.417946,2.917363,1


In [3]:
target = pd.read_csv("proc_data/target.csv", parse_dates=['dt'])
target.head()

,dt,value
0,1999-01-01 00:00:00,0
1,1999-01-01 03:00:00,2
2,1999-01-01 06:00:00,1
3,1999-01-01 09:00:00,2
4,1999-01-01 12:00:00,2


In [4]:
problem = pd.read_csv("proc_data/problem.csv", index_col=0, parse_dates=['dt'])
problem.head()

,Np,Tp,Vp,B_gsm_x,B_gsm_y,B_gsm_z,Bmag
dt,,,,,,,
2015-01-01 00:00:00,1.852,41618.0,455.61,6.098,2.449,-13.415,14.952
2015-01-01 00:01:00,2.181,33781.0,461.07,6.700,1.502,-13.232,14.912
2015-01-01 00:02:00,2.415,34709.0,464.84,7.390,0.523,-12.941,14.928
2015-01-01 00:03:00,2.359,34183.0,470.76,8.011,-0.098,-12.694,15.013
2015-01-01 00:04:00,1.237,32467.0,469.51,7.493,0.410,-13.058,15.064


In [5]:
additional = pd.read_csv("proc_data/additional.csv", index_col=0, parse_dates=['dt'])
additional.head()

,Np,Tp,Vp,B_gsm_x,B_gsm_y,B_gsm_z,Bmag
dt,,,,,,,
2014-06-29 00:01:00,2.416,65701.0,402.18,-4.212,-3.982,2.445,6.298
2014-06-29 00:02:00,2.493,68240.0,399.89,-4.566,-3.473,2.766,6.430
2014-06-29 00:03:00,2.582,85224.0,389.17,-5.456,-2.370,2.837,6.596
2014-06-29 00:04:00,2.686,90925.0,390.59,-5.550,-1.988,2.768,6.519
2014-06-29 00:05:00,2.676,79821.0,392.44,-4.922,-3.073,2.679,6.416


In [21]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

In [14]:
class Dataset(Dataset): 
    """ Diabetes dataset.""" 
    
    # Initialize your data, download, etc. 
    def __init__(self):
        xy = train.iloc[:, 1:].values
        self.len = xy.shape[0] 
        self.x_data = torch.from_numpy(xy[:, :-1]) 
        self.y_data = torch.from_numpy(xy[:, -1]) 
    def __getitem__(self, index): 
        return self.x_data[index], self.y_data[index] 
    def __len__(self): 
        return self.len

In [15]:
dataset = Dataset() 
train_loader = DataLoader(dataset=dataset, batch_size=32, shuffle=True, num_workers=2)

In [23]:
rnn = nn.RNN(10, 20, 2)

In [24]:
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
output, hn = rnn(input, h0)

In [26]:
dataset.x_data.shape

torch.Size([40912, 7])

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [22]:
len(train_loader)

1279